In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [ ]:
data=pd.read_csv("Churn_Modelling.csv")

In [ ]:
data.head()

In [ ]:
data=data.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [ ]:
data

In [ ]:
label_encoder_gender=LabelEncoder()

In [ ]:
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [ ]:
data

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])

In [ ]:
geo_encoder

In [ ]:
geo_encoder.toarray()

In [ ]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

In [ ]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [ ]:
geo_encoded_df

In [ ]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [ ]:
data.head()

In [ ]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

In [ ]:
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [ ]:
data.head()

In [ ]:
X=data.drop('Exited',axis=1)

In [ ]:
y=data['Exited']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train=scaler.fit_transform(X_train)

In [ ]:
X_test=scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

In [ ]:
data

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [ ]:
(X_train.shape[1],)

In [ ]:
model=Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
]
)

In [ ]:
model.summary()

In [ ]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [ ]:
loss

In [ ]:
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [ ]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

In [ ]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

In [ ]:
model.save('model.h5')

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit/20250405-105712